In [1]:
import os

import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt

from torchvision.datasets import ImageFolder
import torch.utils.data as data
import torchvision
from torchvision import transforms
from torchvision import datasets

import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import numpy as np

from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
def load_images(image_size=(32,32), batch_size=64, root="../datasets/MainImageFolder"):

    transform = transforms.Compose([
                    transforms.Resize(image_size),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    train_set = datasets.ImageFolder(root=root, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
    return train_loader 

In [3]:
# train_data = load_images(root='./train/')

In [3]:
# valid_size = 0.2
# batch_size = 64
# num_workers = 0

# train_ld = load_images(root='./train/')

# train_data = train_ld.dataset

# num_train = len(train_data)
# indices = list(range(num_train))
# np.random.shuffle(indices)
# split = int(np.floor(valid_size * num_train))
# train_idx, valid_idx = indices[split:], indices[:split]

# train_sampler = SubsetRandomSampler(train_idx)
# valid_sampler = SubsetRandomSampler(valid_idx)

# train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
#     sampler=train_sampler, num_workers=num_workers)
# valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
#     sampler=valid_sampler, num_workers=num_workers)

# valset= valid_loader.dataset

In [11]:
batch_size = 64
train_ld = load_images(root='./train/')
train_ld1 = load_images(root='./train/')

In [12]:
print(len(train_ld.dataset))
validation_split = 0.3
shuffle_dataset = True
random_seed= 42

22424


In [16]:
# Creating data indices for training and validation splits:
dataset_size = len(train_ld.dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_idx, valid_idx = indices[split:], indices[:split]

def get_same_index(target, label):
    label_indices = []
    
    for i in range(len(target)):
        if target[i][1] == label:
            label_indices.append(i)

    return label_indices

ind = get_same_index(train_ld.dataset.samples,1) # remove c0
sampler_chk1 = torch.utils.data.sampler.SubsetRandomSampler(ind)

# print(len(sampler_chk1))

train_idx1 = list(set(train_idx) - set(sampler_chk1))
# print(len(train_idx1))

valid_idx1 = list(set(valid_idx) - set(sampler_chk1))
len(valid_idx1)

train_data = train_ld.dataset
train_data1 = train_ld1.dataset

train_sampler = SubsetRandomSampler(train_idx1)
valid_sampler = SubsetRandomSampler(valid_idx1)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler)
print(len(train_loader.sampler))
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler)
print(len(valid_loader.sampler))
valid_loader_ooc = torch.utils.data.DataLoader(train_data1, batch_size=batch_size, 
    sampler=sampler_chk1)
print(len(valid_loader_ooc.sampler))

14133
6024
2267


In [17]:
train_data=train_loader.dataset
valid_data=valid_loader.dataset
valset=valid_loader.dataset

In [8]:
# Insipired on LeNet, first proposed in: 
# https://ieeexplore.ieee.org/abstract/document/726791
# PDF: http://www.cs.virginia.edu/~vicente/deeplearning/readings/lecun1998.pdf
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        # Convolutional layers.
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        
        # Linear layers.
        self.fc1 = nn.Linear(64*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Conv1 + ReLU + MaxPooling.
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        
        # Conv2 + ReLU + MaPooling.
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        
        # This flattens the output of the previous layer into a vector.
        out = out.view(out.size(0), -1) 
        
        # Linear layer + ReLU.
        out = F.relu(self.fc1(out))
        # Linear layer + ReLU.
        out = F.relu(self.fc2(out))
        # A final linear layer at the end.
        out = self.fc3(out)
       
        # We will not add Softmax here because nn.CrossEntropy does it.
        # Read the documentation for nn.CrossEntropy.
        return out

In [18]:
NUM_CLASSES = 9


class AlexNet(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 2 * 2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 2 * 2)
        x = self.classifier(x)
        return x

In [22]:
model = AlexNet()

# model = model.cuda()
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.cuda()
correct = 0.0
cum_loss = 0.0

learningRate = 5e-2
optimizer = optim.SGD(model.parameters(), lr = learningRate, 
                      momentum = 0.9, weight_decay = 1e-4)
N = 1
# log accuracies and losses.
train_accuracies = []; val_accuracies = []
train_losses = []; val_losses = []

In [21]:
torch.cuda.is_available()

False

In [24]:
for epoch in range(0, N):
        correct = 0.0
        cum_loss = 0.0

      # Make a pass over the training data.
        model.train()
        total_train=0
        for (i, (inputs, labels)) in enumerate(train_loader):
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                print(inputs.shape)
                labels = labels.cuda()
                print(np.unique(labels.cpu()))
            print(labels.shape)
            total_train+=len(inputs)
            # Forward pass. (Prediction stage)
            scores = model(inputs)
            print(scores.shape)
            loss = loss_fn(scores, labels)

            # Count how many correct in this batch.
            max_scores, max_labels = scores.max(1)
            correct += (max_labels == labels).sum().item()
            cum_loss += loss.item()
            
#             print(i, correct, total_train, correct/total_train)
            
            # Zero the gradients in the network.
            optimizer.zero_grad()

            # Backward pass. (Gradient computation stage)
            loss.backward()

            # Parameter updates (SGD step) -- if done with torch.optim!
            optimizer.step()

            # Parameter updates (SGD step) -- if done manually!
            # for param in model.parameters():
            #   param.data.add_(-learningRate, param.grad)

            # Logging the current results on training.
            if (i + 1) % 100 == 0:
                print('Train-epoch %d. Iteration %05d, Avg-Loss: %.4f, Accuracy: %.4f' % (epoch, i + 1, cum_loss / (i + 1), correct / ((i + 1) * batch_size)))

        train_accuracies.append(correct / total_train)
        train_losses.append(cum_loss / (i + 1))   

        # Make a pass over the validation data.
        correct = 0.0
        cum_loss = 0.0
        model.eval()
        total=0
        for (i, (inputs, labels)) in enumerate(valid_loader):
            if torch.cuda.is_available():
                
                inputs = inputs.cuda()
                labels = labels.cuda()
            
            total += len(inputs)
            # Forward pass. (Prediction stage)
            scores = model(inputs)
            cum_loss += loss_fn(scores, labels).item()

            # Count how many correct in this batch.
            max_scores, max_labels = scores.max(1)
            correct += (max_labels == labels).sum().item()

        val_accuracies.append(correct / total)
        val_losses.append(cum_loss / (i + 1))

        # Logging the current results on validation.
        print('Validation-epoch %d. Avg-Loss: %.4f, Accuracy: %.4f' % 
            (epoch, cum_loss / (i + 1), correct / total))

torch.Size([64])
torch.Size([64, 9])


RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at /pytorch/aten/src/THNN/generic/ClassNLLCriterion.c:93

In [10]:
# Make a pass over the validation data.
correct = 0.0
cum_loss = 0.0
model.eval()
total=0
pred_labels=[]
for (i, (inputs, labels)) in enumerate(valid_loader_ooc):
    inputs = inputs.cuda()
    labels = labels.cuda()

    # Forward pass. (Prediction stage)
    scores = model(inputs)
    cum_loss += loss_fn(scores, labels).item()

    # Count how many correct in this batch.
    max_scores, max_labels = scores.max(1)
    correct += (max_labels == labels).sum().item()
    total+= len(inputs)
    pred_labels+=[max_labels]
#     print(correct, total)

print(correct, len(valid_data))

val_accuracies.append(correct/total)
val_losses.append(cum_loss / (i + 1))

# Logging the current results on validation.
print('Validation-epoch %d. Avg-Loss: %.4f, Accuracy: %.4f' % 
    (epoch, cum_loss / (i + 1), correct/total ))

0.0 22424
Validation-epoch 4. Avg-Loss: 11.5151, Accuracy: 0.0000


In [3]:
pred_np = []
for i in pred_labels:
    pred_np += [i.cpu().numpy()]

NameError: name 'pred_labels' is not defined

In [ ]:
predl=[]
for i in pred_np:
    for j in i:
        predl.append(j)

In [ ]:
pd.DataFrame(predl)

In [ ]:
# len(valid_loader.sampler)
valid_loader.batch_size*len(valid_loader)
valid_loader.batch_sampler.batch_size